After exploring the data, we're going to find of much of it can be relevant for our decision tree. This is a critical point for every Data Science project, since too much train data can easily result in bad model generalisation (accuracy on test/real/unseen observations). 

In this Ipython notebook, I am going to cover Random Forest. Random Forest is a versatile machine learning method capable of performing both regression and classification tasks. It also undertakes dimensional reduction methods, treats missing values, outlier values and other essential steps of data exploration, and does a fairly good job. It is a type of ensemble learning method, where a group of weak models combine to form a powerful model.

Advantages of Random Forest:

1) This algorithm can solve both type of problems i.e. classification and regression and does a decent estimation at both fronts.
      
2) One of benefits of Random forest which excites me most is, <B> the power of handle large data set with higher dimensionality. It can handle thousands of input variables and identify most significant variables so it is considered as one of the dimensionality reduction methods. </B>  Further, the model outputs Importance of variable, which can be a very handy feature (on some random data set).

3) It has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing.

4) It has methods for balancing errors in data sets where classes are imbalanced.

5) The capabilities of the above can be extended to unlabeled data, leading to unsupervised clustering, data views and outlier detection.

6) Random Forest involves sampling of the input data with replacement called as bootstrap sampling. Here one third of the data is not used for training and can be used to testing. These are called the out of bag samples. Error estimated on these out of bag samples is known as out of bag error. Study of error estimates by Out of bag, gives evidence to show that the out-of-bag estimate is as accurate as using a test set of the same size as the training set. Therefore, using the out-of-bag error estimate removes the need for a set aside test set.





In [9]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [10]:
train_df = pd.read_csv('train_cleaned.csv')
test_df  = pd.read_csv('test_cleaned.csv')

id = test_df['Id']
train_df.drop('Id',axis = 1, inplace = True)
test_df.drop('Id',axis = 1 , inplace = True)
y_train = train_df['SalePrice']
x_train = train_df.drop('SalePrice', axis = 1)

test_df.head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Functional_Min2,BldgType_Twnhs,RoofStyle_Mansard,RoofMatl_CompShg,SaleCondition_Partial,GarageCond_Ex,Functional_Maj2,SaleType_New,GarageType_BuiltIn,Exterior2nd_CBlock
0,3.044522,80,9.360741,5,6,1961,1961,0.000000,6.150603,4.976734,...,0,0,0,1,0,0,0,0,0,0
1,3.044522,81,9.565775,6,6,1958,1958,4.691348,6.828712,0.000000,...,0,0,0,1,0,0,0,0,0,0
2,4.110874,74,9.534668,5,5,1997,1998,0.000000,6.674561,0.000000,...,0,0,0,1,0,0,0,0,0,0
3,4.110874,78,9.208238,6,6,1998,1998,3.044522,6.401917,0.000000,...,0,0,0,1,0,0,0,0,0,0
4,4.795791,43,8.518392,8,5,1992,1992,0.000000,5.575949,0.000000,...,0,0,0,1,0,0,0,0,0,0


In [11]:
y_train.head()
feat_labels = train_df.columns.values
#feat_labels
x_train.isnull().values.any()

False

Since, Random Forest is good regression method which has an ability to handle a large amount of high-dimensional data ( here we have ~ 280 columns as features), we are going to construct a function which handles the features and list the features which are important in prediction.Also , I am going to construct a function which fits a given model to the train data and cross validates via K-Fold Cross Validation Scheme 

In [12]:
from sklearn.ensemble import RandomForestRegressor

def printFeature(model,x_train, y_train , performCV = True , cvFolds = 10):
    
    model.fit(x_train,y_train)
    feature_importances = model.feature_importances_
    feat_labels = [ x for x in x_train.columns.values]
    feat_imp = pd.Series(feature_importances,feat_labels).sort_values(ascending=False)
    #print(feat_imp.dtypes)
    #plt.bar(feat_labels[:],feat_imp[:])
    plt.ylabel('Feature Importances')
    #plt.show()
    
def modelFit(model,x_train,y_train,performCV = True , cvFolds = 10):
    
    model.fit(x_train,y_train)
    x_train_predictions = model.predict(x_train)
    
    if performCV == True :
        cv_score_meanSqError = cross_val_score(model,x_train,y_train,cv= cvFolds,scoring = 'mean_squared_error')
        cv_score_r2 = cross_val_score(model,x_train,y_train,cv= cvFolds,scoring = 'r2')
        
    print(" Model Report")
    print(" Mean Squared Error = %.4g" % metrics.mean_squared_error(y_train.values,x_train_predictions))
    print(" R2 score = %.4g" % metrics.r2_score(y_train.values,x_train_predictions))
          
    
    print(" CV Score (Mean Square Error) : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score_meanSqError),np.std(cv_score_meanSqError),np.min(cv_score_meanSqError),np.max(cv_score_meanSqError)))
    print(" CV Score ( R square        ) : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score_r2),np.std(cv_score_r2),np.min(cv_score_r2),np.max(cv_score_r2)))




In [13]:
#defining the model
n_estimators = 1000
random_state = 0
n_jobs = -1
rf = RandomForestRegressor(n_estimators = n_estimators,random_state = random_state,n_jobs = n_jobs) 
#printFeature(rf,x_train,y_train)
modelFit(rf,x_train,y_train,performCV = True, cvFolds = 5)

 Model Report
 Mean Squared Error = 0.002689
 R2 score = 0.9831
 CV Score (Mean Square Error) : Mean - -0.02019539 | Std - 0.002728923 | Min - -0.02373028 | Max - -0.01596594
 CV Score ( R square        ) : Mean - 0.8733335 | Std - 0.0115757 | Min - 0.8545422 | Max - 0.8896669


In [14]:
from sklearn.grid_search import GridSearchCV

<B> Parameter Tunign of the Random Forest Model </B> :

We will use grid search to identify the optimal parameters of our random forest model. Because our training dataset is quite small, we can get away with testing a wider range of hyperparameter values.I will discuss the results of this grid search. I will tune :

n_estimators = Signifies the number of trees used in random forest

and then tune tree specific paramters here :
1) min_samples_split
2) min_smaples_leaf
3) max_depth
4) min_leaf_nodes
5) max_features
6) loss function

In [128]:
#Tuning the no. of trees used in the regresor

param_test1 =  {'n_estimators':np.arange(1000,10000,500)}
rfr = RandomForestRegressor(min_samples_split=2,min_samples_leaf=1,max_depth=8,max_features='sqrt',random_state=10)
gridSearch1 =  GridSearchCV(estimator = rfr, 
                       param_grid = param_test1, scoring='r2',n_jobs=4,iid=False, cv=5)
gridSearch1.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=10,
           verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': array([1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000,
       6500, 7000, 7500, 8000, 8500, 9000, 9500])},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [127]:
gridSearch1.grid_scores_, gridSearch1.best_params_ , gridSearch1.best_score_

([mean: 0.48082, std: 0.01574, params: {'n_estimators': 1000},
  mean: 0.48570, std: 0.01560, params: {'n_estimators': 1500},
  mean: 0.48465, std: 0.01530, params: {'n_estimators': 2000},
  mean: 0.48475, std: 0.01519, params: {'n_estimators': 2500},
  mean: 0.48500, std: 0.01495, params: {'n_estimators': 3000},
  mean: 0.48415, std: 0.01530, params: {'n_estimators': 3500},
  mean: 0.48442, std: 0.01527, params: {'n_estimators': 4000},
  mean: 0.48381, std: 0.01536, params: {'n_estimators': 4500},
  mean: 0.48366, std: 0.01545, params: {'n_estimators': 5000},
  mean: 0.48314, std: 0.01547, params: {'n_estimators': 5500},
  mean: 0.48250, std: 0.01538, params: {'n_estimators': 6000},
  mean: 0.48232, std: 0.01545, params: {'n_estimators': 6500},
  mean: 0.48238, std: 0.01544, params: {'n_estimators': 7000},
  mean: 0.48216, std: 0.01550, params: {'n_estimators': 7500},
  mean: 0.48204, std: 0.01549, params: {'n_estimators': 8000},
  mean: 0.48163, std: 0.01548, params: {'n_estimators':

In [8]:
# Tuning max_depth and min_samples_split

param_test2 =  {'max_depth':np.arange(7,9,1), 'min_samples_split':np.arange(1,11,2)}
rfr = RandomForestRegressor(n_estimators=1500,max_features='sqrt', random_state=0)
gridSearch2 = GridSearchCV(estimator = rfr , param_grid = param_test2 , scoring = 'r2',n_jobs = -1 , iid= False , cv =5)
gridSearch2.fit(x_train,y_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/root/anaconda3/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

...........................................................................
/root/anaconda3/lib/python3.5/runpy.

In [118]:
gridSearch2.grid_scores_, gridSearch2.best_params_ , gridSearch2.best_score_

([mean: 0.70907, std: 0.01400, params: {'max_depth': 5, 'min_samples_split': 200},
  mean: 0.54627, std: 0.01892, params: {'max_depth': 5, 'min_samples_split': 400},
  mean: 0.46493, std: 0.01550, params: {'max_depth': 5, 'min_samples_split': 600},
  mean: -0.00338, std: 0.00473, params: {'max_depth': 5, 'min_samples_split': 800},
  mean: -0.00338, std: 0.00473, params: {'max_depth': 5, 'min_samples_split': 1000},
  mean: 0.71050, std: 0.01405, params: {'max_depth': 7, 'min_samples_split': 200},
  mean: 0.54627, std: 0.01892, params: {'max_depth': 7, 'min_samples_split': 400},
  mean: 0.46493, std: 0.01550, params: {'max_depth': 7, 'min_samples_split': 600},
  mean: -0.00338, std: 0.00473, params: {'max_depth': 7, 'min_samples_split': 800},
  mean: -0.00338, std: 0.00473, params: {'max_depth': 7, 'min_samples_split': 1000},
  mean: 0.71055, std: 0.01413, params: {'max_depth': 9, 'min_samples_split': 200},
  mean: 0.54627, std: 0.01892, params: {'max_depth': 9, 'min_samples_split': 400}

We obtain a really bad score here of 0.71. We will try come back to optimisation of min_sample_split later on . We choose max_depth = 9 as optimum paramter.

In [122]:
param_test1 =  {'min_samples_leaf':np.arange(1,71,6), 'min_samples_split':np.arange(1,200,20)}
rfr = RandomForestRegressor(n_estimators=60,max_depth = 9,max_features='sqrt', random_state=10)
gridSearch2 = GridSearchCV(estimator = rfr , param_grid = param_test1 , scoring = 'r2',n_jobs = -1 , iid= False , cv =5)
gridSearch2.fit(x_train,y_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/root/anaconda3/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

...........................................................................
/root/anaconda3/lib/python3.5/runpy.

In [120]:
gridSearch2.grid_scores_, gridSearch2.best_params_ , gridSearch2.best_score_

([mean: 0.80172, std: 0.01473, params: {'min_samples_leaf': 3, 'min_samples_split': 60},
  mean: 0.77277, std: 0.01373, params: {'min_samples_leaf': 3, 'min_samples_split': 100},
  mean: 0.74661, std: 0.01298, params: {'min_samples_leaf': 3, 'min_samples_split': 140},
  mean: 0.72263, std: 0.01311, params: {'min_samples_leaf': 3, 'min_samples_split': 180},
  mean: 0.78965, std: 0.00935, params: {'min_samples_leaf': 13, 'min_samples_split': 60},
  mean: 0.76161, std: 0.01268, params: {'min_samples_leaf': 13, 'min_samples_split': 100},
  mean: 0.74197, std: 0.01123, params: {'min_samples_leaf': 13, 'min_samples_split': 140},
  mean: 0.71896, std: 0.01309, params: {'min_samples_leaf': 13, 'min_samples_split': 180},
  mean: 0.77485, std: 0.01285, params: {'min_samples_leaf': 23, 'min_samples_split': 60},
  mean: 0.75107, std: 0.01385, params: {'min_samples_leaf': 23, 'min_samples_split': 100},
  mean: 0.73340, std: 0.01352, params: {'min_samples_leaf': 23, 'min_samples_split': 140},
  mean